# Импорт библиотек

In [1]:
import yaml
import pandas as pd
import os
import gpxpy
import gpxpy.gpx
import re
from transliterate import translit
from datetime import datetime, timedelta
import calendar
from isdayoff import DateType, ProdCalendar
from tqdm import tqdm_notebook
import joblib
import requests

# Файл с параметрами

In [2]:
config_path = '../config/params.yml'
config = yaml.load(open(config_path), Loader=yaml.FullLoader)

preproc = config['preprocessing']
training = config['traning']
evaluate = config['evaluate']

# Обработка данных данных

In [3]:
def identifiers_auto(path: str, sep: str, encoding: str) -> pd.DataFrame:
    """
    Загрузка индификаторов автомобилей
    :param path: путь до файла с индификаторами авто
    :param sep: разделитель для csv файла
    :param encoding: кодировка файла  
    :return: DataFrame
    """
    return pd.read_csv(path, sep=sep, encoding=encoding)


def dict_csv_model(identifiers_auto: pd.DataFrame, path: str, model_auto: str,
                   id_auto: str) -> dict:
    """
    Создание словаря со списками (Модель:[список CSV треков под определенную модель авто]
    :param identifiers_auto: DataFrame c индификаторами автомобилей
    :param path: путь до папки с треками
    :param model_auto: название столбца с моделями авто из identifiers_auto
    :param id_auto: название столбца с id авто из identifiers_auto
    :return: DataFrame
    """

    # Создание словаря со списками (Модель:[список CSV треков под определенную модель авто]
    dict_csv_model = {}

    # Перебор по моделям авто
    for model in identifiers_auto[model_auto].unique():

        # Генератор списка содержащий список CSV треков под определенную модель авто
        list_csv = [
            filename for filename in os.listdir(path)
            if int(filename.split('_')[0]) in list(identifiers_auto[
                identifiers_auto[model_auto] == model][id_auto].unique())
        ]

        # Добавление списка с словарь с ключом МОДЕЛЬ
        if len(list_csv) != 0:
            dict_csv_model[str(model)] = list_csv.copy()
            list_csv.clear()
        else:
            list_csv.clear()

    return dict_csv_model


def data_df_track(path: str, dict_csv_model: dict) -> pd.DataFrame:
    """
    Создание DataFrame с данными
    :param path: путь до папки с треками
    :param dict_csv_model: словарь со списками (Модель:[список CSV треков под определенную модель авто]
    :return: DataFrame
    """

    # Создание конечного DataFrame
    data = pd.DataFrame()

    # Перебор моделей авто в цикле
    for key in dict_csv_model.keys():

        # Перебор файлов треков в цикле
        for gpx_csv in dict_csv_model[key]:
            with open(preproc['path_open'] + gpx_csv, 'r',
                      encoding='utf-8') as gpx_file:
                gpx = gpxpy.parse(gpx_file)

            # Создание списка для словаря для преобразования в DataFrame
            initial_data = []

            # Перебор информации в треке в цикле
            for track in gpx.tracks:
                for segment in track.segments:
                    for count, point in enumerate(segment.points):

                        # Добавление в список словаря с контрольными точками
                        initial_data.append({
                            'id':
                            gpx_csv.split('.')[0],
                            'model':
                            str(
                                re.sub(
                                    ' ', '_',
                                    translit(key,
                                             language_code='ru',
                                             reversed=True))),
                            'progression':
                            count + 1,
                            'latitude':
                            point.latitude,
                            'longitude':
                            point.longitude,
                            'elevation':
                            point.elevation,
                            'date_and_time':
                            datetime.strftime(point.time, "%Y-%m-%d %H:%M:%S")
                        })
            # Закрытие файла с треком
            gpx_file.close()

            # Преобразование списка со славорем в DataFrame
            initial_data_df = pd.DataFrame(initial_data)

            # Создание DataFrame где будет содержаться информации о первой точки в отрезке
            start = initial_data_df.copy(deep=True).drop(
                initial_data_df.index[-1], axis=0).reset_index().drop('index',
                                                                      axis=1)
            # Переименовывание столбцов
            start.columns = [
                'id', 'model', 'start_progression', 'start_latitude',
                'start_longitude', 'start_elevation', 'start_date_and_time'
            ]

            # Создание DataFrame где будет содержаться информации о последней точки в отрезке
            end = initial_data_df.copy(deep=True).drop(
                initial_data_df.index[0],
                axis=0).reset_index().drop(['id', 'index', 'model'], axis=1)

            # Переименовывание столбцов
            end.columns = [
                'end_progression', 'end_latitude', 'end_longitude',
                'end_elevation', 'end_date_and_time'
            ]

            # Соединение двух DataFrame'ов
            full = pd.concat([start, end], axis=1)

            # Соединение двух DataFrame'ов
            data = (pd.concat([data, full], axis=0, ignore_index=True))

    return data


def transform_types(data: pd.DataFrame,
                    change_type_columns: dict) -> pd.DataFrame:
    """
    Преобразование признаков в заданный тип данных
    :param data: датасет
    :param change_type_columns: словарь с признаками и типами данных
    :return: DataFrame
    """
    return data.astype(change_type_columns, errors="raise")


def days_week(data: pd.DataFrame, column_data: str) -> list:
    """
    Добавление дня недели
    :param data: датасет
    :param colum_data: название столбца с датой
    :return: список с днями недели
    """
    return [calendar.day_name[day.weekday()] for day in data[column_data]]


def type_days(data: pd.DataFrame, name_column_time: str) -> object:
    """
    Добавление характеристик автомобиля
    :param data: датасет
    :param name_columns_time: название столбца с датой с типом datetime64[ns]
    :return: датасет с типом дня
    """
    type_days = []
    for day in data[name_column_time]:
        type_days.append(day.date())
    type_days = pd.Series(list(set(type_days))).astype('datetime64[ns]')
    types_days = {}
    
    # Определение рабочено, не рабочего и пред не рабочего дня
    for day in type_days:
        response = requests.get('https://isdayoff.ru/{}{}{}'.format(
            day.year, day.strftime("%m"), day.strftime("%d")))
        if response.status_code == 200:
            if response.json() == 0:
                rs = requests.get('https://isdayoff.ru/{}{}{}'.format(
                    (day + timedelta(days=1)).year,
                    (day + timedelta(days=1)).strftime("%m"),
                    (day + timedelta(days=1)).strftime("%d")))
                if rs.json() == 0:
                    types_days[day.date()] = 'working_day'

                else:
                    types_days[day.date()] = 'before_non-working_day'
            else:
                types_days[day.date()] = 'not_working_day'
        else:
            break
    return (data[name_column_time].apply(lambda x: x.date())).map(types_days)


def car_characteristics(data: pd.DataFrame, name_column_model: str,
                        characteristics: dict) -> list:
    """
    Добавление характеристик автомобиля
    :param data: датасет
    :param name_columns_model: название столбца с моделями авто
    :param characteristics: словарь с характеристиками авто
    :return: список характеристиками
    """
    return (data[name_column_model].apply(lambda x: x)).map(characteristics)


def pickup_hour(data: pd.DataFrame, name_column_time: str) -> list:
    """
    Добавление часа когда происходила запись точки в трек
    :param data: датасет
    :param  name_column_time: название столбца с временем в формат 
     времени datetime64[ns]
    :return: список с часами
    """
    return data[name_column_time].apply(lambda x: x.hour)


def rmsle_loss(y_true, y_pred):
    grad = 2 * (np.log(y_pred + 1) - np.log(y_true + 1)) / (y_pred + 1)
    hess = (-2 * np.log(y_pred + 1) + 2 * np.log(y_true + 1) + 2) / (y_pred +
                                                                     1)**2
    return grad, hess

In [4]:
def pipeline_preprocess(path_id: str, sep: str, encoding: str, path_track: str,
                        model_auto: str, id_auto: str,
                        change_type_columns: dict, colum_data: str, name_columns_model:str,
                        feature_auto: dict, name_column_time: str, drop_columns: list) -> pd.DataFrame:
    """
    Пайплайн по предобработке данных
    :param path_id: путь до файла с индификаторами авто
    :param sep: разделитель для csv файла
    :param encoding: кодировка файла
    :param path_track: путь до папки с треками
    :param model_auto: название столбца с моделями авто identifiers_auto
    :param id_auto: название столбца с id авто identifiers_auto
    :param change_type_columns: словарь с признаками и типами данных
    :param colum_data: название столбца с датой
    :param name_columns_model: название столбца с моделями авто
    :param feature_auto: словарь с характеристиками авто
    :param  name_column_time: название столбца с временем в формат 
     времени datetime64[ns]
    :param  drop_columns: список с удаляемыми столбцами
     
    
    :return: DataFrame
    """

    auto = identifiers_auto(path_id, sep=sep, encoding=encoding)

    id_csv_model = dict_csv_model(auto, path_track, model_auto, id_auto)

    data = data_df_track(path_track, id_csv_model)

    data = transform_types(data, change_type_columns)

    data['days_of_the_week'] = days_week(data, name_column_time)
    
    data['day_type'] = type_days(data, name_column_time)
    
    for feature in feature_auto.keys():
        data[feature] = car_characteristics(data, name_columns_model, feature_auto[feature])
        
    data = data[(data['type_auto'] !=  'ABG') & (data['type_auto'] !='APT')]
    
    data['pickup_hour'] = pickup_hour(data, name_column_time)
    
    # Удаление не нужны колонок
    data = data.drop(drop_columns, axis=1)
    
    # Замена типа данных с object на category
    for col in data.select_dtypes(object).columns:
        data[col] = data[col].astype('category')
        

    return data

In [5]:
data_proc_test = pipeline_preprocess(
    preproc['id_auto'], ';', 'cp1251', preproc['path_open'], 'МОДЕЛЬ', 'ID',
    preproc['change_type_columns'], 'start_date_and_time', 'model',
    preproc['car_characteristics'], 'start_date_and_time',
    evaluate['drop_columns'])

In [6]:
data_proc_test

,model,days_of_the_week,day_type,max_speed_km/h,full_mass_kg,engine_power_l_s,type_auto,pickup_hour
0,ASA_(IVECO_MAGIRUS_RW-DAILY_65C18D),Thursday,working_day,100,12000,194,ASA,16
1,ASA_(IVECO_MAGIRUS_RW-DAILY_65C18D),Thursday,working_day,100,12000,194,ASA,16
2,ASA_(IVECO_MAGIRUS_RW-DAILY_65C18D),Thursday,working_day,100,12000,194,ASA,16
3,ASA_(IVECO_MAGIRUS_RW-DAILY_65C18D),Thursday,working_day,100,12000,194,ASA,16
4,ASA_(IVECO_MAGIRUS_RW-DAILY_65C18D),Thursday,working_day,100,12000,194,ASA,16
...,...,...,...,...,...,...,...,...
84548,ASA_(GAZ-_32590L),Friday,before_non-working_day,90,3500,106,ASA,14
84549,ASA_(GAZ-_32590L),Friday,before_non-working_day,90,3500,106,ASA,14
84550,ASA_(GAZ-_32590L),Friday,before_non-working_day,90,3500,106,ASA,14
84551,ASA_(GAZ-_32590L),Friday,before_non-working_day,90,3500,106,ASA,14


# Evaluate

In [7]:
model = joblib.load(training['model_path_lgbm_optuna'])

In [8]:
data_proc_test['speed_km/h'] = model.predict(data_proc_test)

In [9]:
data_proc_test

,model,days_of_the_week,day_type,max_speed_km/h,full_mass_kg,engine_power_l_s,type_auto,pickup_hour,speed_km/h
0,ASA_(IVECO_MAGIRUS_RW-DAILY_65C18D),Thursday,working_day,100,12000,194,ASA,16,30.036041
1,ASA_(IVECO_MAGIRUS_RW-DAILY_65C18D),Thursday,working_day,100,12000,194,ASA,16,30.036041
2,ASA_(IVECO_MAGIRUS_RW-DAILY_65C18D),Thursday,working_day,100,12000,194,ASA,16,30.036041
3,ASA_(IVECO_MAGIRUS_RW-DAILY_65C18D),Thursday,working_day,100,12000,194,ASA,16,30.036041
4,ASA_(IVECO_MAGIRUS_RW-DAILY_65C18D),Thursday,working_day,100,12000,194,ASA,16,30.036041
...,...,...,...,...,...,...,...,...,...
84548,ASA_(GAZ-_32590L),Friday,before_non-working_day,90,3500,106,ASA,14,21.287566
84549,ASA_(GAZ-_32590L),Friday,before_non-working_day,90,3500,106,ASA,14,21.287566
84550,ASA_(GAZ-_32590L),Friday,before_non-working_day,90,3500,106,ASA,14,21.287566
84551,ASA_(GAZ-_32590L),Friday,before_non-working_day,90,3500,106,ASA,14,21.287566
